In [1]:
import json

from collections import Counter

In [2]:
with open('../public/static/bike_routes.geojson') as fh:
    data = json.loads(fh.read())

In [3]:
categories = Counter()
lengths = Counter()

for feature in data['features']:
    categories[feature['properties']['bike_network_category']] += 1
    if feature['geometry']:
        lengths[len(feature.get('geometry', {}).get('coordinates'))] += 1

In [4]:
categories

Counter({'Proposed Bike+ - Non-Arterial': 3076,
         'Proposed Bike+ - Arterial': 1503,
         'Proposed Bike+ - Upgrade, Arterial': 1464,
         'Existing Bike+ - Non-Arterial': 804,
         'Existing Bike+ - Arterial': 525,
         'Existing Multi-Use Trail': 337,
         'Non-Bike+ Planned': 250,
         'Non-Bike+': 197,
         'Proposed Multi-Use Trail': 175,
         '': 136})

In [21]:
!pip install pyhigh

     |████████████████████████████████| 19.8MB 16.2MB/s eta 0:00:01
  Running setup.py install for pyhigh ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
data['features'][0]

{'type': 'Feature',
 'id': 1,
 'geometry': {'type': 'LineString',
  'coordinates': [[-122.308954277912, 47.604327437898],
   [-122.307649047488, 47.6043216339769]]},
 'properties': {'OBJECTID': 1,
  'id': 23677,
  'artclass_desc': 'Not Designated',
  'stname_ord': 'E ALDER ST',
  'bike_network_category': 'Proposed Bike+ - Non-Arterial'}}

In [15]:
from pyhigh import get_elevation

In [29]:
get_elevation(lat=47.604327437898, lon=-122.308954277912) * 3.28084

314.96064

In [28]:
get_elevation(lat=47.6043216339769, lon=-122.307649047488) * 3.28084

321.52232

In [27]:
get_elevation(lat=47.5541164, lon=-122.2896943) * 3.28084

144.35696

In [16]:
pairs = {}
min_elev = 0
max_elev = 0

for feature in data['features']:
    if not feature['geometry']:
        continue
    idx = 0
    while idx < len(feature['geometry']['coordinates']) - 1:
        origin = feature['geometry']['coordinates'][idx]
        dest = feature['geometry']['coordinates'][idx + 1]
        origin_elev = get_elevation(lat=origin[1], lon=origin[0])
        dest_elev = get_elevation(lat=dest[1], lon=dest[0])
        pair = f"{origin} - {dest}"
        pairs[pair] = dest_elev - origin_elev
        max_elev = max(pairs[pair], max_elev)
        min_elev = min(pairs[pair], min_elev)
        idx += 1

In [18]:
max_elev

np.float64(73.0)

In [21]:
for p in pairs:
    print(p, pairs[p])
    break

[-122.308954277912, 47.604327437898] - [-122.307649047488, 47.6043216339769] 2.0


In [23]:
output = {
    'type': 'FeatureCollection',
    'crs': { 'type': 'name', 'properties': {'name': 'EPSG:4326'}},
    'features': []
}
for p in pairs:
    if pairs[p] > 0:
        color = '#ff0000'
    else:
        color = '#00ff00'
    output['features'].append({
        "type": "Feature",
        "id": 1,
        "geometry": {
            "type": "LineString",
            "coordinates":[p.split(' - ')[0], p.split(' - ')[1]]
        },
        "properties": {'color': color}
    })

In [24]:
with open(

{'type': 'FeatureCollection',
 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}},
 'features': [{'type': 'Feature',
   'id': 1,
   'geometry': {'type': 'LineString',
    'coordinates': ['[-122.308954277912, 47.604327437898]',
     '[-122.307649047488, 47.6043216339769]']},
   'properties': {'color': '#ff0000'}},
  {'type': 'Feature',
   'id': 1,
   'geometry': {'type': 'LineString',
    'coordinates': ['[-122.310746309136, 47.6685386099947]',
     '[-122.309604606436, 47.6685412488534]']},
   'properties': {'color': '#ff0000'}},
  {'type': 'Feature',
   'id': 1,
   'geometry': {'type': 'LineString',
    'coordinates': ['[-122.38540827951, 47.6886998216099]',
     '[-122.385416670102, 47.6905822606564]']},
   'properties': {'color': '#ff0000'}},
  {'type': 'Feature',
   'id': 1,
   'geometry': {'type': 'LineString',
    'coordinates': ['[-122.314686924417, 47.5672369450935]',
     '[-122.314704223832, 47.5665782057277]']},
   'properties': {'color': '#ff0000'}},
  {'type': 'F

In [25]:
with open('../public/static/bike_routes_heights.geojson', 'w+') as fh:
    fh.write(json.dumps(output))